# Programa auxiliar para generar imágenes de Carteles (con su correspondiente XML) combinando al azar las imágenes de Letras independientes disponibles en el Drive

0) Configurar los parámetros para la ejecución:

In [0]:
# cantidad de imágenes combinadas a generar
cantImagGen = 500

# prefijo nombre imágenes a generar
prefijoImagGen = 'cartel'

# cantidad mínima/máxima de imágenes disponibles seleccionadas para incluir por imagen a generar combinada
cantMinImagSelecc = 2
cantMaxImagSelecc = 10

# tamaño base de la imagen a generar
anchoImagGen = 128
altoImagGen = 128

# pixeles para ajustar coordenadas para XML
xDesplXML = 10
yDesplXML = 5

# determina si genera imágenes para entrenar o probar ( 'train' o 'test' )
tipoImagSelecc = 'train' 

# directorio local donde están disponibles las imágenes fuente de imágenes (están divididos por clases)
pathFuente = 'gdrive/My Drive/IA/demoConvNet-Letras/Letras/' + tipoImagSelecc

# directorio local donde se guardarán las nuevas imágenes generadas (las pone todas juntas, luego decide si son de entrenamiento o prueba)
pathDestino = 'gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/' 

print ("Parámetros definidos.")

Parámetros definidos.


1) Cargar librerías:

In [0]:
import os
import os.path
from os.path import isfile, join

import random
from random import seed
from random import randint
from random import choice
seed(7)

import numpy as np

from PIL import Image
import csv

import xml.etree.cElementTree as ET

print ("Librerías cargadas.")

Librerías cargadas.


2) Montar el Drive:

In [0]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


3) Levantar la lista de imágenes disponibles para combinar:

In [0]:
# variables auxiliares
all_images_array = []
all_dirs = os.listdir(pathFuente)

# crea el directorio destino, si es necesario
if not os.path.isdir(pathDestino):
  os.makedirs(pathDestino)

# procesa cada directorio
allFileNames = []
for each_dir in all_dirs:
  # levanta la lista de imágenes de cada letra
  auxFileNames = os.listdir( pathFuente + '/' + each_dir )

  # le agrega el path completo  
  for fn in auxFileNames:
    allFileNames.append( pathFuente + '/' + each_dir + '/' + fn )

# muestra resumen
print('> Clases disponibles: ', all_dirs)
print('> Total de imágenes disponibles: ', len(allFileNames))


> Clases disponibles:  ['4', '5', '2', '7', '9', '6', '8', '3', '1', 'B', 'A', '0', 'E', 'F', 'D', 'C', 'G', 'K', 'I', 'J', 'H', 'M', 'N', 'L', 'O', 'Q', 'P', 'R', 'V', 'U', 'Z', 'W', 'X', 'S', 'Y', 'T']
> Total de imágenes disponibles:  29563


4) Combinar al azar las imágenes disponibles para generar las nuevas:

In [0]:
# obtiene el ID Máximo de imagen generada para empezar a numerar desde ahí
id = 0
list_of_files =  [f for f in os.listdir(pathDestino) if isfile(join(pathDestino, f))] 
if (len(list_of_files) > 0):
  for strfile in list_of_files:
    auxiVec = strfile.split("_",2)
    if len(auxiVec)>2:
      num = int(auxiVec[1]) 
      id = num if num > id else id  
print('> ID inicial', prefijoImagGen, id, '\n')

# desordena la lista de imágenes disponible
np.random.shuffle(allFileNames)

# Genera la cantidad de imágenes indicadas en los parámetros
list_clases = []
auxiCantMostrarProgreso = cantImagGen//5
print('\n** Comienza proceso de generación de imágenes **\n')
for _ in range( cantImagGen ):

  # define el nombre de la imagen a generar
  id = id + 1
  nombreImagComb = prefijoImagGen + '_' + str(id) + '_'  
  
  #  determina al azar la cantidad de imágenes para utilizar como base  
  seleccImag_cant = randint(cantMinImagSelecc, cantMaxImagSelecc)

  # genera una imagen en blanco de un tamaño acorde a la cantidad de imágenes a utilizar 
  # (por ahora combina todo con una línea horizontal)
  nuevaImgAnchoTotal = anchoImagGen*seleccImag_cant
  nuevaImgAltoTotal = altoImagGen
  nuevaImg  = Image.new('RGB', (nuevaImgAnchoTotal, nuevaImgAltoTotal))

  # combina la cantidad determinada al azar de imágenes disponibles
  auxiAnchoDesp = 0
  auxiAltoDesp = 0
  xml_annotation_list = []
  for _ in range( seleccImag_cant ):
      
      # selecciona al azar una imagen de la lista
      auxiSelFN = choice(np.array(allFileNames))

      # define el tipo de la clase (LETRA o NÚMERO) de la imagen seleccionada
      auxiLetraFuente = auxiSelFN[len(pathFuente)+1]
      if auxiLetraFuente in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        auxiSelClase = 'N'
      else:
        auxiSelClase = 'L'

      # concatena la letra al nombre y el tipo de clase a la lista
      nombreImagComb = nombreImagComb + auxiLetraFuente
      list_clases.append(auxiSelClase)

      # agrega la imagen seleccionada al azar 
      # obteniendo también las coordenadas para el XML
      auxiSelImg = Image.open(auxiSelFN)
      nuevaImg.paste(auxiSelImg, (auxiAnchoDesp, auxiAltoDesp))    
      auxi_xmin = auxiAnchoDesp + xDesplXML
      auxi_ymin = auxiAltoDesp + yDesplXML
      auxiAnchoDesp = auxiAnchoDesp + auxiSelImg.size[0] 
      auxi_xmax = auxiAnchoDesp - xDesplXML
      auxi_ymax = auxiAltoDesp + auxiSelImg.size[1] - yDesplXML
      xml_annotation_list.append( str(auxi_xmin) + ',' + str(auxi_ymin) + ',' 
                                + str(auxi_xmax) + ',' + str(auxi_ymax) + ',' + auxiSelClase )

  # define nombre de archivos
  nombreImagCombFN = nombreImagComb + '.png'
  xmlnombreImagCombFN = nombreImagComb + '.xml'
  
  # graba la nueva imagen 
  nuevaImg.save( pathDestino + nombreImagCombFN, "PNG")

  # genera el XML de la nueva imagen   
  xml_annotation = ET.Element('annotation')
  ET.SubElement(xml_annotation, 'folder').text = pathDestino
  ET.SubElement(xml_annotation, 'filename').text = nombreImagCombFN
  ET.SubElement(xml_annotation, 'path').text = pathDestino + nombreImagCombFN

  xml_source = ET.SubElement(xml_annotation, 'source')
  ET.SubElement(xml_source, 'database').text = 'Unknown'

  xml_size = ET.SubElement(xml_annotation, 'size')
  ET.SubElement(xml_size, 'width').text = str(nuevaImgAnchoTotal)
  ET.SubElement(xml_size, 'height').text = str(nuevaImgAltoTotal)
  ET.SubElement(xml_size, 'depth').text = str(3)

  ET.SubElement(xml_annotation, 'segmented').text = '0'

  for annot in xml_annotation_list:
    tmp_annot = annot.split(',')
    xmin, ymin, xmax, ymax, label = tmp_annot[0], tmp_annot[1], tmp_annot[2], tmp_annot[3], tmp_annot[4]

    object = ET.SubElement(xml_annotation, 'object')
    ET.SubElement(object, 'name').text = label
    ET.SubElement(object, 'pose').text = 'Unspecified'
    ET.SubElement(object, 'truncated').text = '0'
    ET.SubElement(object, 'difficult').text = '0'

    xml_bndbox = ET.SubElement(object, 'bndbox')
    ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
    ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
    ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
    ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)

  xml_tree = ET.ElementTree(xml_annotation)
  xml_tree.write( pathDestino + xmlnombreImagCombFN )

  # muestra cada tanto para que haga más rápido que si muestra todas
  if (id%auxiCantMostrarProgreso)==0: 
    print('- IMAGEN GENERADA: ', nombreImagComb, '(', nombreImagCombFN, ' + ', xmlnombreImagCombFN, ')' )


print('\n** Proceso de generación de imágenes terminado **\n')
print("\n> Se generan imágenes hasta el ID ", id)
print('\n> Clases usadas para generar imágenes: ', list_clases)

> ID inicial cartel 0 


** Comienza proceso de generación de imágenes **

- IMAGEN GENERADA:  cartel_100_YEYC ( cartel_100_YEYC.png  +  cartel_100_YEYC.xml )
- IMAGEN GENERADA:  cartel_200_KKDPFB4ZU ( cartel_200_KKDPFB4ZU.png  +  cartel_200_KKDPFB4ZU.xml )
- IMAGEN GENERADA:  cartel_300_D5H5KH ( cartel_300_D5H5KH.png  +  cartel_300_D5H5KH.xml )
- IMAGEN GENERADA:  cartel_400_3Y ( cartel_400_3Y.png  +  cartel_400_3Y.xml )
- IMAGEN GENERADA:  cartel_500_P31 ( cartel_500_P31.png  +  cartel_500_P31.xml )

** Proceso de generación de imágenes terminado **


> Se generan imágenes hasta el ID  500

> Clases usadas para generar imágenes:  ['L', 'L', 'L', 'L', 'N', 'N', 'N', 'N', 'N', 'L', 'N', 'N', 'L', 'L', 'L', 'N', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'L', 'N', 'N', 'L', 'L', 'L', 'L', 'N', 'L', 'L', 'N', 'L', 'L', 'L', 'N', 'L', 'L', 'N', 'N', 'N', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'N', 'L', 'L', 'N', 'L', 'L', 'L', 'N', 'L', 'L', 'N', 'L', 'L', 'L', 'L',

In [0]:
# graba un archivo CSV con las clases que fueron levantadas y usadas (saca duplicadas y ordena)
list_clases_Uni = list(dict.fromkeys(list_clases))
list_clases_Uni.sort()

print('- Clases levantadas sin duplicados: ', list_clases_Uni)
with open(pathDestino + 'clases.csv', mode='w') as csvfile:
    csvfileWriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvfileWriter.writerow(  list_clases_Uni )
print('> Archivo de clases grabado en ', pathDestino + 'clases.csv')


- Clases levantadas sin duplicados:  ['L', 'N']
> Archivo de clases grabado en  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/clases.csv
